# **Data & Model Downloading**

In [ ]:
!pip install sklearn-crfsuite

In [ ]:
!pip install pythainlp[attacut,ner,benchmarks]

     |████████████████████████████████| 11.1MB 2.6MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 1.4MB 49.8MB/s 
     |████████████████████████████████| 1.3MB 36.9MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 276kB 41.4MB/s 
     |████████████████████████████████| 481kB 36.7MB/s 
     |████████████████████████████████| 747kB 48.7MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434672 sha256=ea09ed02ec6eb225641c29b777b676478f251e70027e9aab15dc7a3c98e90fb4
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
  Created wheel for fire: filename=fire-0.3.1-py2.py3-none-any.whl size=111005 sha256=f1d0f77a5ba85c0c3dae980beaea13d4aef4381eda78458f9a601f74afe0aa79
  Stored in directory: /root/.cache/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
  Created wheel for pyyaml: filename=PyYAML

In [ ]:
!pip install git+git://github.com/rkcosmos/deepcut.git

  Cloning git://github.com/rkcosmos/deepcut.git to /tmp/pip-req-build-6rz3dn54
  Running command git clone -q git://github.com/rkcosmos/deepcut.git /tmp/pip-req-build-6rz3dn54
  Created wheel for deepcut: filename=deepcut-0.7.0.0-cp36-none-any.whl size=2011586 sha256=4a7f9093746c0b3c5220fd651b663b25b00990fb414e011d3143908efc858745
  Stored in directory: /tmp/pip-ephem-wheel-cache-p93v7r5f/wheels/1d/a7/f3/c9794455c7ece304440686e3e7b2ad895e1d0e7dec27d49a55
Successfully built deepcut


In [ ]:
!gdown --id 19koix8oiQJYfp0N94BxyyngBtq6KhXV8

Downloading...
From: https://drive.google.com/uc?id=19koix8oiQJYfp0N94BxyyngBtq6KhXV8
To: /content/contest3-master.zip
3.32MB [00:00, 106MB/s]


In [ ]:
!unzip contest3-master.zip

Archive:  contest3-master.zip
212fe7432fbd71aeb19600d2947d5e864f10c4b0
replace contest3-master/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip contest3-master/data.zip 

Archive:  contest3-master/data.zip
replace __MACOSX/._thai_ner_dev_set? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# **Importing Modules**

In [ ]:
import sklearn_crfsuite
import sklearn_crfsuite.metrics
import re
import requests
import json
from pythainlp import word_tokenize, Tokenizer
from pythainlp.tag import pos_tag
from pythainlp.tag.named_entity import ThaiNameTagger

# **Hadling Raw Files**

## **Load Raw Files**

In [ ]:
#Load Test Files
fn=[]
for n in range(160,190):
  fn.append('corpus0'+str(n)+'-utf8.txt')
test={}
for files in fn:
  dr='thai_ner_test_set_no_answers/'+files
  with open(dr, mode='r', encoding='utf8') as rf:
    test[files]=rf.read()

FileNotFoundError: ignored

In [ ]:
#Load No-ANS dev Files
fn=[]
for n in range(150,160):
  fn.append('corpus0'+str(n)+'-utf8.txt')
na_dev={}
for files in fn:
  dr='thai_ner_dev_set_no_answers/'+files
  with open(dr, mode='r', encoding='utf8') as rf:
    na_dev[files]=rf.read()

FileNotFoundError: ignored

In [ ]:
#Load dev Files w/ ANS
fn=[]
for n in range(150,160):
  fn.append('corpus0'+str(n)+'-utf8.txt')
dev={}
for files in fn:
  dr='thai_ner_dev_set/'+files
  with open(dr, mode='r', encoding='utf8') as rf:
    dev[files]=rf.read()

In [ ]:
#Load Train Files w/ ANS
fn=[]
for n in range(1,100):
  fn.append('corpus00'+str(n)+'-utf8.txt')
for n in range(100,150):
  fn.append('corpus0'+str(n)+'-utf8.txt')
train={}
for files in fn:
  dr='thai_ner_train/'+files
  with open(dr, mode='r', encoding='utf8') as rf:
    train[files]=rf.read()

## **Decode Files w/ ANS**

In [ ]:
def ind_e(s):
  if re.match('.+<abb>.+?</abb>.+',s):
    if re.match('.+</abb>(.+)</.+?Ind>',s):
      ind=abb_e(s)+re.search('<.+?Ind>\s*<abb>.+</abb>(.+)</.+?Ind',s).group(1)
    else:
      ind=abb_e(s)
  else:
    ind=re.search('<.+?Ind>(.+?)</.+?Ind>',s).group(1)
  return(ind)

def abb_e(s):
  abb= ''.join(re.findall('<abb>(.+?)</abb>',s))
  return(abb)

def title_e(s):
  tt= re.sub('<.+?>','',re.search('<title>(.+?)</title>',s).group(1))
  return(tt)

In [ ]:
#Extract dev Files w/ ANS
d_dev={}
for k,i in dev.items():
  org=re.findall('(<orgName Id=.+?>.+?</orgName>)',i)
  pers=re.findall('(<persName Id=.+?>.+?</persName>)',i)
  place=re.findall('(<placeName Id=.+?>.+?</placeName>)',i)
  dc={}
  all_org=[]
  all_pers=[]
  all_place=[]
  for item in org:
    if re.match('<orgName Id=.+?>\s*<orgInd>.+</orgInd>.+</orgName>',item):
      if re.match('<orgName Id=.+?>\s*<orgInd>.+</orgInd>.+<orgIndB>.+</orgIndB>\s*</orgName>',item):
        all_org.append(ind_e(item)+re.search('.+</orgInd>(.+)<orgIndB>',item).group(1)+re.search('.+<orgIndB>(.+)</orgIndB>',item).group(1))
      else:
        all_org.append(ind_e(item)+re.search('.+</orgInd>(.+)</orgName>',item).group(1))
    elif re.match('<orgName Id=.+?>\s*<abb>.+</abb>\s*</orgName>',item):
      all_org.append(abb_e(item))
    else:
      all_org.append(re.search('<orgName Id=.+?>(.+)</orgName>',item).group(1))
  for item in pers:
    if re.match('<persName Id=.+?>\s*<title>.+</title>.+</persName>',item):
      all_pers.append(title_e(item)+re.search('.+</title>(.+)</persName>',item).group(1))
    else:
      all_pers.append(re.search('<persName Id=.+?>(.+)</persName>',item).group(1))
  for item in place:
    if re.match('<placeName Id=.+?>\s*<placeInd>.+</placeInd>.+</placeName>',item):
      all_place.append(ind_e(item)+re.search('.+</placeInd>(.+)</placeName>',item).group(1))
    elif re.match('<placeName Id=.+?>\s*<abb>.+</abb>\s*</placeName>',item):
      all_place.append(abb_e(item))
    else:
      all_place.append(re.search('<placeName Id=.+?>(.+)</placeName>',item).group(1))
  dc['org']=list(set(all_org))
  dc['pers']=list(set(all_pers))
  dc['place']=list(set(all_place))
  d_dev[k]=dc



In [ ]:
#extract Train Files w/ ANS
d_train={}
for k,i in train.items():
  org=re.findall('(<orgName Id=.+?>.+?</orgName>)',i)
  pers=re.findall('(<persName Id=.+?>.+?</persName>)',i)
  place=re.findall('(<placeName Id=.+?>.+?</placeName>)',i)
  dc={}
  all_org=[]
  all_pers=[]
  all_place=[]
  for item in org:
    if re.match('<orgName Id=.+?>\s*<orgInd>.+</orgInd>.+</orgName>',item):
      if re.match('<orgName Id=.+?>\s*<orgInd>.+</orgInd>.+<orgIndB>.+</orgIndB>\s*</orgName>',item):
        all_org.append(ind_e(item)+re.search('.+</orgInd>(.+)<orgIndB>',item).group(1)+re.search('.+<orgIndB>(.+)</orgIndB>',item).group(1))
      else:
        all_org.append(ind_e(item)+re.search('.+</orgInd>(.+)</orgName>',item).group(1))
    elif re.match('<orgName Id=.+?>\s*<abb>.+</abb>\s*</orgName>',item):
      all_org.append(abb_e(item))
    else:
      all_org.append(re.search('<orgName Id=.+?>(.+)</orgName>',item).group(1))
  for item in pers:
    if re.match('<persName Id=.+?>\s*<title>.+</title>.+</persName>',item):
      all_pers.append(title_e(item)+re.search('.+</title>(.+)</persName>',item).group(1))
    else:
      all_pers.append(re.search('<persName Id=.+?>(.+)</persName>',item).group(1))
  for item in place:
    if re.match('<placeName Id=.+?>\s*<placeInd>.+</placeInd>.+</placeName>',item):
      all_place.append(ind_e(item)+re.search('.+</placeInd>(.+)</placeName>',item).group(1))
    elif re.match('<placeName Id=.+?>\s*<abb>.+</abb>\s*</placeName>',item):
      all_place.append(abb_e(item))
    else:
      all_place.append(re.search('<placeName Id=.+?>(.+)</placeName>',item).group(1))
  dc['org']=list(set(all_org))
  dc['pers']=list(set(all_pers))
  dc['place']=list(set(all_place))
  d_train[k]=dc


In [ ]:
for a,b in zip(t_train['corpus0138-utf8.txt'],

## **Tokenize**

In [ ]:
#dev
t_dev={}
for k,i in na_dev.items():
  t_dev[k]=word_tokenize(re.sub('\n','',i),engine="deepcut",keep_whitespace=False)

In [ ]:
#train
t_train={}
na_train={}
for k,i in train.items():
  na_train[k]=re.sub('<.+?>','',i)
for k,i in na_train.items():
  t_train[k]=word_tokenize(re.sub('\n','',i),engine="deepcut",keep_whitespace=False)

In [ ]:
#test
t_test={}
for k,i in test.items():
  t_test[k]=word_tokenize(re.sub('\n','',i),engine="deepcut",keep_whitespace=False)

## **Sentence Tagging**

### **NER Tagging**

In [ ]:
#dev
ner_dev=[]
for k,i in t_dev.items():
  nd=[]
  for t in i:
    if t in d_dev[k]['pers']:
      nd.append('pers')
    elif t in d_dev[k]['place']:
      nd.append('place')
    elif t in d_dev[k]['org']:
      nd.append('org')
    else:
      nd.append('o')
  ner_dev.append(nd)

In [ ]:
#train
ner_train=[]
for k,i in t_train.items():
  nd=[]
  for t in i:
    if t in d_train[k]['pers']:
      nd.append('pers')
    elif t in d_train[k]['place']:
      nd.append('place')
    elif t in d_train[k]['org']:
      nd.append('org')
    else:
      nd.append('o')
  ner_train.append(nd)

### **POS Tagging**

In [ ]:
#dev
xdev=[]
for k,t in t_dev.items():
  xdev.append(pos_tag(t, corpus='orchid'))

In [ ]:
#train
xtrain=[]
for k,t in t_train.items():
  xtrain.append(pos_tag(t, corpus='orchid'))

In [ ]:
#test
xtest=[]
for k,t in t_test.items():
  xtest.append(pos_tag(t, corpus='orchid'))

## **Dictionary Creating**

In [ ]:
with open('contest3-master/name.json','r') as na:
  name=json.load(na)
  na=str(name)
name_list=[]
for k,i in name.items():
  name_list.append(list(i))

In [ ]:
with open('contest3-master/place.txt','r') as pl:
  place=str(pl.read())
  place_list=place.split('\n')

In [ ]:
with open('contest3-master/company.txt','r') as cm:
  compa=str(cm.read())
  com_list=compa.split('\n')

In [ ]:
with open('contest3-master/organization.txt','r') as org:
  o=str(org.read())
  org_list=o.split('\n')

# **Features Engineering**

In [ ]:
#Word
def pos_features(i, tokens):
  pos_f={}
  if i >0:
    pos_f.update({'t-1 pos': tokens[i-1][1]})
  pos_f.update({'t0 pos': tokens[i][1]})
  if i < len(tokens)-1:
    pos_f.update({'t+1 pos': tokens[i+1][1]})
  return pos_f
    
def pos_conjunctive_features(i, tokens):
  if i < len(tokens)-1:
    if i >0:
      return {'pos_c':[ tokens[i-1][1], tokens[i][1], tokens[i+1][1]]}
    else: 
      return {'pos_c':[tokens[i][1], tokens[i+1][1]]}
  else:
    return {'pos_c':[ tokens[i-1][1], tokens[i][1]]}

def word_features(i, tokens):
  if i == 0:
    return {'word_current': tokens[i][0], 'word_next': tokens[i+1][0]}
  elif i == len(tokens) - 1:
    return {'word_current': tokens[i][0], 'word_previous': tokens[i-1][0]}
  else:
    return {'word_current': tokens[i][0], 'word_next': tokens[i+1][0], 'word_previous': tokens[i-1][0]}

def inclu(i, tokens):
  if re.match('.*(จ.|จังหวัด).*',tokens[i][0]):
    return({'included':'Province'})
  elif re.match('.*(ประเทศ).*',tokens[i][0]):
    return({'included':'Country'})
  elif re.match('.*(บริษัท|บ\.|จำกัด|มหาชน).*',tokens[i][0]):
    return({'included':'Company'})
  elif re.match('.*(นาย|นาง|นางสาว|นส.).*',tokens[i][0]):
    return({'included':'Person'})
  else:
     return({'included':'nothing'})



def in_dict(i,tokens):
  q=''.join(['.*',tokens[i][0],'.*'])
  if q in na:
    return({'in_dict':'Name'})
  elif q in place:
    return({'in_dict':'Place'})
  elif q in compa:
    return({'in_dict':'Company'})
  elif q in o:
    return({'in_dict':'Organization'})
  else:
    return({'in_dict':'Not_in_dict'})

In [ ]:
def in_dict(i,tokens):
  q=''.join(['.*',tokens[i][0],'.*'])
  if re.match(q,na):
    return({'in_dict':'Name'})
  elif re.match(q,place):
    return({'in_dict':'Place'})
  elif re.match(q,compa):
    return({'in_dict':'Company'})
  elif re.match(q,o):
    return({'in_dict':'Organization'})
  else:
    return({'in_dict':'Not_in_dict'})

"{'name': ['อภิสิทธิ์', 'อดิศักดิ์', 'อักขระ', 'อริศรา', 'อมร', 'อมรรัตน์', 'อนันต์', 'อนันตชัย', 'อาณัติ', 'อัญชลี', 'อัญชลีพร', 'อนุชา', 'อาภัสรา', 'อภิชาต', 'อภิชาติ', 'อภิรักษ์', 'อภิศักดิ์', 'อภิญญา', 'อารี', 'อารีพงศ์', 'อารง', 'อาทิตย์', 'อรุณศรี', 'อัษรา', 'อัษฎา', 'อรรถสิทธิ์', 'บัณฑิตา', 'บัญญัติ', 'บุญศรี', 'บุญรัตน์', 'บุญเยี่ยม', 'บุญยง', 'บุษราคัม', 'บุศวรรณ', 'ชัยอนันต์', 'ชัยพร', 'จักรี', 'เฉลิม', 'ฉลวย', 'ชนะ', 'ชาญชัย', 'ชนินทร์', 'ชนิภรณ์', 'จันทร์สิริ', 'เชาวน์', 'ชาติชาย', 'ชาติ', 'จตุพล', 'ชิด', 'ชมเกดุ', 'ชุ่ม', 'ชวน', 'จุลภาส', 'ชูวิทย์', 'ดนัย', 'แดง', 'ดนุพล', 'ดวงใจ', 'ดวงกมล', 'เดือนเพ็ญ', 'ดุสิต', 'เอกใหม่', 'เอกพงษ์', 'ฟุ้งฟ้า', 'กานดา', 'แก้วเก้า', 'กมล', 'กนกพร', 'กระสินธ์', 'หทัย', 'จันทนา', 'จารุวรรณ', 'จินตหรา', 'จินทนา', 'เจือ', 'กาญจนา', 'เกษม', 'ขจี', 'ขนิษฐา', 'คลัง', 'ขวัญจิต', 'กิติ', 'กล้าหาญ', 'กฤษฏิ์', 'กุมโชค', 'กุญชนิตา', 'กุลพันธ์', 'ขวัญชัย', 'ขวัญใจ', 'มาลี', 'มะลิ', 'มณี', 'มาณี', 'มานิจ', 'มโน', 'มณู', 'มนู', 'มานิตย์', 'มารวย', 'มงคล'

In [ ]:
#START END POS
def pos_features(i, tokens):
  pos_f={}
  if i >0:
    pos_f.update({'t-1 pos': tokens[i-1][1]})
  elif i==0:
    pos_f.update({'t-1 pos': 'START'})
  pos_f.update({'t0 pos': tokens[i][1]})
  if i < len(tokens)-1:
    pos_f.update({'t+1 pos': tokens[i+1][1]})
  elif i == len(tokens)-1:
    pos_f.update({'t+1 pos': 'END'})
  return pos_f
    
def pos_conjunctive_features(i, tokens):
  if i < len(tokens)-1:
    if i >0:
      return {'pos_c':[ tokens[i-1][1], tokens[i][1], tokens[i+1][1]]}
    else: 
      return {'pos_c':['START',tokens[i][1], tokens[i+1][1]]}
  else:
    return {'pos_c':[ tokens[i-1][1], tokens[i][1],'END']}
    
def word_features(i, tokens):
  w_f={}
  if i >0:
    w_f.update({'t-1 w': tokens[i-1][0]})
  elif i ==0:
    w_f.update({'t-1 w':'START' })
  w_f.update({'t0 w': tokens[i][0]})
  if i < len(tokens)-1:
    w_f.update({'t+1 w': tokens[i+1][0]})
  elif i == len(tokens)-1:
    w_f.update({'t+1 w': 'END'})
  return w_f

EVALUATION

In [ ]:
def featurize_one_sentence(tokens, feature_function_list):
    feature_dict_seq = []
    for i in range(len(tokens)): # ทุก token
        feature_dict = {}
        for feature_fn in feature_function_list: # ทุก feature function
            feature_dict.update(feature_fn(i, tokens))
        feature_dict_seq.append(feature_dict)
    return feature_dict_seq
    
def train_model(X_train, Y_train, X_test, Y_test, feature_function_list):
  Xtrain_data = [featurize_one_sentence(token_list, feature_function_list) for token_list in X_train]
  Xtest_data = [featurize_one_sentence(token_list, feature_function_list) for token_list in X_test]
  crf = sklearn_crfsuite.CRF()
  crf.fit(Xtrain_data, Y_train)
  pred = crf.predict(Xtest_data)
  return [pred,Xtrain_data,Xtest_data]

In [ ]:
pred,aa,bb=train_model(X_train=xtrain,Y_train=ner_train, X_test=xdev, Y_test=ner_dev, feature_function_list=[pos_features,pos_conjunctive_features,word_features,inclu,in_dict])

In [ ]:
#For dev
fn=[]
for n in range(150,160):
  fn.append('corpus0'+str(n)+'-utf8.txt')
ans={}
for n in range(len(fn)):
  org=[]
  place=[]
  pers=[]
  for tn in range(len(pred[n])):
    if pred[n][tn] =='org':
      org.append(xdev[n][tn][0])
    elif pred[n][tn] == 'place':
      place.append(xdev[n][tn][0])
    elif pred[n][tn] == 'pers':
      pers.append(xdev[n][tn][0])
  ans[fn[n]]={}
  ans[fn[n]]['org']=list(set(org))
  ans[fn[n]]['place']=list(set(place))
  ans[fn[n]]['pers']=list(set(pers))

  
with open('my_answer.json','w')as gg:
  json.dump(ans,gg)

In [ ]:
#For test
fn=[]
for n in range(160,190):
  fn.append('corpus0'+str(n)+'-utf8.txt')
ans={}
for n in range(len(fn)):
  org=[]
  place=[]
  pers=[]
  for tn in range(len(pred[n])):
    if pred[n][tn] =='org':
      org.append(xtest[n][tn])
    elif pred[n][tn] == 'place':
      place.append(xtest[n][tn])
    elif pred[n][tn] == 'pers':
      pers.append(xtest[n][tn])
  ans[fn[n]]={}
  ans[fn[n]]['org']=list(set(org))
  ans[fn[n]]['place']=list(set(place))
  ans[fn[n]]['pers']=list(set(pers))

  
with open('my_answer.json','w')as gg:
  json.dump(ans,gg)

In [ ]:
ans

{'corpus0150-utf8.txt': {'org': ['โกล์ด ฟิวเจอร์ส',
   'ทองคำ',
   'ตลท.',
   'บริษัท ตลาดอนุพันธ์ (ประเทศไทย'],
  'pers': ['นางเกศรา มัญชุศรี'],
  'place': []},
 'corpus0151-utf8.txt': {'org': ['มหาวิทยาลัยหอการค้าไทย',
   'คลัง',
   'เอสเอ็มอีแบงก์'],
  'pers': ['นายธนวรรธน์ พลวิชัย',
   'นายอภิสิทธิ์ เวชชาชีวะ',
   'นายกอร์ปศักดิ์ สภาวสุ',
   'นายกรณ์',
   'นางเสาวณีย์ ไทยรุ่งโรจน์',
   'นายกรณ์ จาติกวณิช'],
  'place': ['ไทย']},
 'corpus0152-utf8.txt': {'org': ['คลัง', 'สศค.'],
  'pers': ['นายสมชัย สัจจพงษ์', 'นายกรณ์ จาติกวณิช', 'นายสมชัย'],
  'place': ['ไทย']},
 'corpus0153-utf8.txt': {'org': ['บริษัททรูมูฟ จำกัด'],
  'pers': ['นายศุภชัย เจียรวนนท์'],
  'place': ['ไทย']},
 'corpus0154-utf8.txt': {'org': ['คลัง',
   'กรอ.',
   'สศค.',
   'สศช.',
   'ธนาคารแห่งประเทศไทย',
   'กระทรวงการคลัง'],
  'pers': ['นายอภิสิทธิ์ เวชชาชีวะ',
   'นายกรณ์ จาติกวณิช',
   'สำนักเลขาธิการนายก'],
  'place': ['ไทย']},
 'corpus0155-utf8.txt': {'org': [],
  'pers': ['นายสืบศักดิ์ นวจินดา', 'นายสุนทร อรุ

In [ ]:
sklearn_crfsuite.metrics.flat_accuracy_score(ner_dev, pred)

0.9838616714697407

In [ ]:
!python3 contest3-master/evaluate2.py thai_ner_dev_set_answers.json my_answer.json

       PRECISION  RECALL     F1 SUPPORT
place     1.0000  0.2941 0.4545      17
org       0.7000  0.3559 0.4719      59
pers      0.9500  0.7600 0.8444      25
MACRO     0.8833  0.4700 0.5903     101
MICRO     0.8182  0.4455 0.5769     101


In [ ]:
pred,aa,bb=train_model(X_train=xtrain,Y_train=ner_train, X_test=xdev, Y_test=ner_dev, feature_function_list=[pos_features,pos_conjunctive_features])
#For dev
fn=[]
for n in range(150,160):
  fn.append('corpus0'+str(n)+'-utf8.txt')
ans={}
for n in range(len(fn)):
  org=[]
  place=[]
  pers=[]
  for tn in range(len(pred[n])):
    if pred[n][tn] =='org':
      org.append(xdev[n][tn][0])
    elif pred[n][tn] == 'place':
      place.append(xdev[n][tn][0])
    elif pred[n][tn] == 'pers':
      pers.append(xdev[n][tn][0])
  ans[fn[n]]={}
  ans[fn[n]]['org']=list(set(org))
  ans[fn[n]]['place']=list(set(place))
  ans[fn[n]]['pers']=list(set(pers))

  
with open('my_answer.json','w')as gg:
  json.dump(ans,gg)

!python3 contest3-master/evaluate2.py thai_ner_dev_set_answers.json my_answer.json

       PRECISION  RECALL     F1 SUPPORT
place     0.7500  0.1765 0.2857      17
org       0.6667  0.0339 0.0645      59
pers      1.0000  0.0400 0.0769      25
MACRO     0.8056  0.0835 0.1424     101
MICRO     0.7500  0.0594 0.1101     101


In [ ]:
pred,aa,bb=train_model(X_train=xtrain,Y_train=ner_train, X_test=xdev, Y_test=ner_dev, feature_function_list=
                       [pos_features,pos_conjunctive_features,inclu])
#For dev
fn=[]
for n in range(150,160):
  fn.append('corpus0'+str(n)+'-utf8.txt')
ans={}
for n in range(len(fn)):
  org=[]
  place=[]
  pers=[]
  for tn in range(len(pred[n])):
    if pred[n][tn] =='org':
      org.append(xdev[n][tn][0])
    elif pred[n][tn] == 'place':
      place.append(xdev[n][tn][0])
    elif pred[n][tn] == 'pers':
      pers.append(xdev[n][tn][0])
  ans[fn[n]]={}
  ans[fn[n]]['org']=list(set(org))
  ans[fn[n]]['place']=list(set(place))
  ans[fn[n]]['pers']=list(set(pers))

  
with open('my_answer.json','w')as gg:
  json.dump(ans,gg)

!python3 contest3-master/evaluate2.py thai_ner_dev_set_answers.json my_answer.json

       PRECISION  RECALL     F1 SUPPORT
place     0.7500  0.3529 0.4800      17
org       0.6667  0.0339 0.0645      59
pers      0.7333  0.4400 0.5500      25
MACRO     0.7167  0.2756 0.3648     101
MICRO     0.7308  0.1881 0.2992     101


In [ ]:
pred,aa,bb=train_model(X_train=xtrain,Y_train=ner_train, X_test=xdev, Y_test=ner_dev, feature_function_list=
                       [pos_features,pos_conjunctive_features,word_features,inclu,in_dict])
#For dev
fn=[]
for n in range(150,160):
  fn.append('corpus0'+str(n)+'-utf8.txt')
ans={}
for n in range(len(fn)):
  org=[]
  place=[]
  pers=[]
  for tn in range(len(pred[n])):
    if pred[n][tn] =='org':
      org.append(xdev[n][tn][0])
    elif pred[n][tn] == 'place':
      place.append(xdev[n][tn][0])
    elif pred[n][tn] == 'pers':
      pers.append(xdev[n][tn][0])
  ans[fn[n]]={}
  ans[fn[n]]['org']=list(set(org))
  ans[fn[n]]['place']=list(set(place))
  ans[fn[n]]['pers']=list(set(pers))

  
with open('my_answer.json','w')as gg:
  json.dump(ans,gg)

!python3 contest3-master/evaluate2.py thai_ner_dev_set_answers.json my_answer.json


       PRECISION  RECALL     F1 SUPPORT
place     1.0000  0.2941 0.4545      17
org       0.7000  0.3559 0.4719      59
pers      0.9500  0.7600 0.8444      25
MACRO     0.8833  0.4700 0.5903     101
MICRO     0.8182  0.4455 0.5769     101


In [ ]:
len(xtrain)

149

In [ ]:
len(pred)

30

In [ ]:
xtrain[1][5]

('ขอบ', 'NCMN')

In [ ]:
pred,aa,bb=train_model(X_train=xtrain,Y_train=ner_train, X_test=xtest, Y_test=ner_dev, feature_function_list=
                       [pos_features,pos_conjunctive_features,word_features,inclu,in_dict])
#For dev
fn=[]
for n in range(160,190):
  fn.append('corpus0'+str(n)+'-utf8.txt')
ans={}
for n in range(len(fn)):
  org=[]
  place=[]
  pers=[]
  for tn in range(len(pred[n])):
    if pred[n][tn] =='org':
      org.append(xtest[n][tn][0])
    elif pred[n][tn] == 'place':
      place.append(xtrain[n][tn][0])
    elif pred[n][tn] == 'pers':
      pers.append(xtrain[n][tn][0])
  ans[fn[n]]={}
  ans[fn[n]]['org']=list(set(org))
  ans[fn[n]]['place']=list(set(place))
  ans[fn[n]]['pers']=list(set(pers))

  
with open('thai_ner_test_set_predictions.json','w')as gg:
  json.dump(ans,gg)



In [ ]:
pred,aa,bb=train_model(X_train=xtrain,Y_train=ner_train, X_test=xdev, Y_test=ner_dev, feature_function_list=
                       [word_features,inclu,in_dict])
#For dev
fn=[]
for n in range(150,160):
  fn.append('corpus0'+str(n)+'-utf8.txt')
ans={}
for n in range(len(fn)):
  org=[]
  place=[]
  pers=[]
  for tn in range(len(pred[n])):
    if pred[n][tn] =='org':
      org.append(xdev[n][tn][0])
    elif pred[n][tn] == 'place':
      place.append(xdev[n][tn][0])
    elif pred[n][tn] == 'pers':
      pers.append(xdev[n][tn][0])
  ans[fn[n]]={}
  ans[fn[n]]['org']=list(set(org))
  ans[fn[n]]['place']=list(set(place))
  ans[fn[n]]['pers']=list(set(pers))

  
with open('my_answer.json','w')as gg:
  json.dump(ans,gg)

!python3 contest3-master/evaluate2.py thai_ner_dev_set_answers.json my_answer.json


       PRECISION  RECALL     F1 SUPPORT
place     1.0000  0.2941 0.4545      17
org       0.7273  0.2712 0.3951      59
pers      0.9500  0.7600 0.8444      25
MACRO     0.8924  0.4418 0.5647     101
MICRO     0.8511  0.3960 0.5405     101
